In [1]:
update_from = '2022-19' #YYYY-WW
update_to = '2022-20' #YYYY-WW
# Actualizar mes a mes el path
path = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\MEX\Precios\Data\2022\05 Mayo\Output'
filename = '2022 Mayo_20220512_1452.xlsx'

# Libraries

In [2]:
from datetime import datetime
import pandas as pd
import pyodbc
import json
import warnings
warnings.filterwarnings("ignore")
pd.io.formats.excel.ExcelFormatter.header_style = None

# Parameters

In [3]:
today = datetime.today()
year = today.year
week = today.isocalendar()[1]

# Connection

In [4]:
conn = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_MasterOp;'
                     'Trusted_Connection=yes;')

## Queries

In [5]:
qprices = '''SELECT  P.Año
       ,P.Semana
       ,C.PaisNombre
       --,C.GrpID
       ,C.GrpNombre
       ,P.CadID
       --,C.CadNombre
       ,P.ProPstID
       ,EP.ProPstCodBarras
       ,P.[Precio Base]
  FROM Gnm_MasterOp.dbo.GnmPreciosxCadenaHist AS P with(nolock)
  LEFT JOIN Gnm_MasterOp.dbo.vw_EstructuraProductosInternacional ep ON P.ProPstID=EP.ProPstID
  LEFT JOIN Gnm_MasterOp.dbo.vw_EstructuraClientesSegPTVTotal AS C
  ON P.CadID = C.CadID
  WHERE C.PaisNombre LIKE '%méx%' AND P.Año = {0} AND P.Semana = {1} AND EP.ProPstCodBarras IS NOT NULL
  '''

In [6]:
qproducts = '''SELECT DISTINCT
          PR.ProPstCodBarras,
          PR.ProPstNombre AS Descripción,
          PR.ProPstID
FROM Gnm_MasterOp.dbo.GnmPresentacionesProd AS PR
RIGHT JOIN Gnm_MasterOp.dbo.GnmTiposComProd AS PA
ON PA.TipoComProd = PR.TipoComProd
WHERE PA.TipoComNombre LIKE '%canal%' AND PR.ProPstCodBarras != 0  AND PR.ProPstCodBarras IS NOT NULL
ORDER BY ProPstID DESC;
'''

In [7]:
qweeks = '''SELECT SemID
        ,SemAnio
        ,SemNumero
  FROM [Gnm_MasterOp].[dbo].[CatSemanas]
  WHERE (SemAnio BETWEEN {0} AND {1}) AND (SemNumero BETWEEN {2} AND {3})
'''

# Load Data

## DWH

## Price from DWH

In [8]:
qprices = qprices.format(year, week)

dprices = pd.read_sql(qprices, conn)

## ProPstIDs

In [9]:
dproducts = pd.read_sql(qproducts, conn)
dproducts['ProPstCodBarras'] = dproducts['ProPstCodBarras'].map(int).map(str)

## Weeks to change prices

In [10]:
qweeks = qweeks.format(
    update_from.split('-')[0], 
    update_to.split('-')[0],
    int(update_from.split('-')[1]),
    int(update_to.split('-')[1])
)

In [11]:
dweeks = pd.read_sql(qweeks, conn)

## Catalogue to find each client

In [12]:
path_json = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\MEX\Precios\Data\Catálogos\catalogo_clientes.json'
with open (path_json, 'r', encoding='utf-8') as file:
    catalogue_clients = json.load(file)

catalogue = pd.DataFrame(catalogue_clients)

## New Prices

In [13]:
ean_to_update = pd.read_excel(path + '\\' + filename)
ean_to_update['ProPstCodBarras'] = ean_to_update['ProPstCodBarras'].map(int).map(str)
ean_to_update.rename({'Precio':'Precio Base'}, axis=1, inplace=True)

# Main

In [14]:
cols = [
    'CadID'
    ,'ProPstID'
    ,'Precio Base'
    ,'AñoIni'
    ,'SemIni'
    ,'AñoFin'
    ,'SemFin'
    ,'IDSemIni'
    ,'IDSemFin'
]

Acá obtenemos los precios a nivel Cadena y ProPstID

In [15]:
price_cadid = pd.DataFrame()
for cadid in catalogue['CadID'].unique():
    client = catalogue.loc[catalogue['CadID']==cadid, 'Nombre Catálogo']
    catalog_target = ean_to_update[ean_to_update['Cliente']==client.squeeze()]
    aux = catalog_target.merge(dproducts, on='ProPstCodBarras', how='left')[['ProPstID', 'Precio Base']]
    aux['CadID'] = cadid
    price_cadid = pd.concat([price_cadid, aux])
price_cadid = price_cadid[price_cadid['ProPstID'].notnull()]
price_cadid.reset_index(drop=True, inplace=True)

In [16]:
price_cadid['ProPstID'] = price_cadid['ProPstID'].map(int)

Checaremos qué precios hay que actualizar cruzando con lo que tenemos en el DWH

In [17]:
price_cadid['ID'] = price_cadid['CadID'].map(str) + '-' + price_cadid['ProPstID'].map(str)
dprices['ID'] = dprices['CadID'].map(str) + '-' + dprices['ProPstID'].map(str)

In [18]:
check_prices = dprices.merge(price_cadid, on='ID', how='left')
check_prices.dropna(axis=0, inplace=True) #Puede que sean skus no activos

In [19]:
def differents_prices(new_price, last_price):
    diff = new_price - last_price
    return abs(diff)>0.1

In [20]:
check_prices = check_prices[check_prices.apply(lambda row: differents_prices(row['Precio Base_x'], row['Precio Base_y']), axis=1)]

In [21]:
to_update = check_prices[['CadID_y', 'ProPstID_y', 'Precio Base_y']].copy()

In [22]:
to_update['AñoIni'] = dweeks.loc[0, 'SemAnio']
to_update['SemIni'] = dweeks.loc[0, 'SemNumero']
to_update['AñoFin'] = dweeks.loc[dweeks.index[-1], 'SemAnio']
to_update['SemFin'] = dweeks.loc[dweeks.index[-1], 'SemNumero']
to_update['IDSemIni'] = dweeks.loc[0, 'SemID']
to_update['IDSemFin'] = dweeks.loc[dweeks.index[-1], 'SemID']

In [23]:
to_update.rename({
    'CadID_y':'CadID', 
    'ProPstID_y':'ProPstID', 
    'Precio Base_y':'Precio Base'
    }, 
    axis=1, 
    inplace=True)

In [24]:
to_update['CadID'] = to_update['CadID'].map(int)

In [25]:
to_update.isnull().sum()

CadID          0
ProPstID       0
Precio Base    0
AñoIni         0
SemIni         0
AñoFin         0
SemFin         0
IDSemIni       0
IDSemFin       0
dtype: int64

In [26]:
filename_export = 'Actualizar Precios MEX_{0}.xlsx'
filename_export = filename_export.format(today.strftime('%Y%m%d-%H%M'))

In [27]:
to_update[cols].sort_values(['CadID', 'ProPstID']).to_excel(
    path + '\\' + filename_export, 
    index=False, 
    sheet_name='{0}_{1}'.format(year, str(week).zfill(2))
)